# Sleep IMU Example

<div class="alert alert-block alert-info">
This example notebook illustrates how to import IMU data collected during sleep and compute sleep endpoints.
</div>

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

from nilspodlib import Dataset

import biopsykit as bp
import biopsykit.signals as signals
import biopsykit.signals.utils as su
import biopsykit.sleep as sleep
import biopsykit.signals.imu as imu

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')
sns.set_theme(style='ticks')

## Load Example Dataset

In [ ]:
data, fs = bp.example_data.get_sleep_imu_example()
data.head()

### Alternatively: Load your own dataset

Replace the path by the path pointing to your IMU file

In [ ]:
data, fs = bp.io.load_dataset_nilspod("../example_data/sleep_imu/sleep_imu_sample_01.bin")
data.head()

## Plot IMU Data

In [ ]:
fig, ax = bp.sleep.plotting.sleep_imu_plot(data, downsample_factor=5*fs, figsize=(10,5))

## Compute Sleep Endpoints

The Sleep Endpoint Prediction Pipeline (which is performed in `bp.sleep.sleep_endpoints.predict_pipeline()`) consists of the following steps:
1. **Wear Detection**: Detects wear/non-wear blocks. Cut data to the major wear block (i.e., the time period with the longest wearing time)
1. **Compute Acivity Samples**: Converts 3-axis accelerometer data into activity samples like they are delivered by Actigraph devices
1. **Major Rest Period**: Estimates the major rest period (MRP) (i.e., an estimation about the time actually spent in bed)
1. **Sleep/Wake Detection**: Classifies sleep/wake periods based on the "Cole/Kripke" Algorithm (Cole and Kripke, 1993)
1. **Sleep Endpoint Computation**: Computes the following Sleep Endpoints based on sleep/wake periods and MRP:
    1. `date`: Date of the sleep recording **Note**: For easier data handling, this field will return the *previous* day if the MRP started after 12 a.m. (and before 12 p.m.), i.e. the subject went to bed after midnight. Hence, one always night always spans *two days*.
    1. `total_sleep_time`: Total sleep timie in the major rest period (in minutes)
    1. `sleep_onset`: Time when *first sleep phase* started
    1. `wake_onset`: Time when *last sleep phase* ended
    1. `major_rest_period_start`: Start of MRP, i.e. beginning of the time spent in bed quietly
    1. `major_rest_period_end`: End of MRP, i.e. end of the time spent in bed quietly
    1. `sleep_onset_latency`: Time needed to fall asleep, i.e. difference between *sleep onset* and *MRP start*
    1. `getup_latency`: Time needed to get out of bed, i.e. difference between *wake onset* and *MRP end*
    1. `sleep_efficiency`: Ratio of the *total time asleep*, i.e. the time actually spent sleeping and the total time between *sleep onset* and *wake onset* in Percent
    1. `wake_after_sleep_onset`: Duration of time awake after *sleep onset* (and before *wake onset*) (in minutes)
    1. `sleep_bouts`: Bouts spent sleeping
    1. `wake_bouts`: Bouts spent wake
    1. `number_wake_bouts`: Number of wake bouts
    
The results are stored in a dictionary:

In [ ]:
sleep_results = sleep.sleep_endpoints.predict_pipeline(data, sampling_rate=fs, sleep_wake_scale_factor=0.1)
sleep_endpoints = sleep_results['sleep_endpoints']
sleep_endpoints

In [ ]:
data = sleep.sleep_endpoints.cut_to_wear_block(data, sleep_results['major_wear_block'])

In [ ]:
df = sleep.sleep_endpoints.endpoints_as_df(sleep_endpoints, "01")
df

## Plot IMU Data with Sleep Endpoints

In [ ]:
fig, ax = bp.sleep.plotting.sleep_imu_plot(data, sleep_endpoints=sleep_endpoints, downsample_factor=5*fs, figsize=(10,5))

### Export Sleep Endpoints

In [ ]:
sleep.io.save_sleep_endpoints("test.csv", df)